In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# So code is automatically reloaded when saved in different modules

In [17]:
import os
import pickle
import numpy as np
import tensorflow as tf
# from scipy import stats
import tensorflow as tf
# from sklearn.metrics import roc_auc_score
# import sklearn.metrics
from sklearn import metrics
import scipy
import matplotlib.pyplot as plt
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
import pprint
sys.path.append('src/taggerSystem/')
from my_data_util_2 import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [18]:
# data_train = "data/icd9NotesDataTable_train.csv"
# data_valid = "data/icd9NotesDataTable_valid.csv"
data_train = "data/smallIcd9NotesDataTable_train.csv"
# data_train = "data/smallIcd9NotesDataTable_train_2.csv"
data_valid = "data/smallIcd9NotesDataTable_valid.csv"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50
maxAllowedNoteLength = 3
codeIdx = 9
textIdx = 6

In [19]:
ICDCODELIST = []
ICDCODEDICT = {}# this allows us to map codes to integer values.
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx,
    helperLoadPath = None)#'results/temp/features.pkl')
outputPath = 'results/temp/'
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
# helper.save(outputPath)# token2id and max length saved to output_path
# with open(os.path.join(outputPath, "icdMapping.pkl"), "wb") as f:
#     pickle.dump(helper.icdDict, f)

In [20]:
print(helper.icdDict)
print(helper.tok2id)
print(helper.max_length)

{'4240': 5, '456': 0, '45': 1, '123': 3, '486': 4, '18': 2}
{'cat': 2, 'horse': 3, 'CASE:aa': 5, 'CASE:aA': 8, 'deadpool': 4, '</s>': 9, 'CASE:Aa': 6, 'dog': 1, 'CASE:AA': 7, 'the': 5, 'UUUNKKK': 10, '<s>': 11}
3


In [21]:
embeddings[:,1:5]

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.49164701,  0.614438  , -0.149609  , -0.285045  ],
       [-0.780038  ,  0.430511  , -0.208709  , -0.55492598],
       [ 0.032049  , -0.34206101, -0.74521202, -0.88928097],
       [ 0.004422  ,  0.006439  , -0.004394  ,  0.009065  ],
       [ 1.00277305, -1.406829  , -0.016482  ,  0.459856  ],
       [ 0.82947177, -0.34992629,  0.72440302,  1.19216001],
       [ 0.48994979,  1.22273219,  1.38936615, -0.2411315 ],
       [-2.26171994, -0.04729586, -0.02004942,  0.39720514],
       [ 2.02874303, -2.45117092,  0.355793  , -0.85604501],
       [ 0.81293803,  0.66100264,  1.65423679,  1.18584061],
       [-0.236247  , -0.65228099,  0.40685201,  0.84960198],
       [ 0.5996834 , -0.28633684, -1.08432508, -1.70849454]], dtype=float32)

In [22]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = "data/smallIcd9NotesDataTable_train_2.csv", data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx,
    helperLoadPath = 'results/temp/')
outputPath = 'results/temp/'
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
# helper.save(outputPath)# token2id and max length saved to output_path
# with open(os.path.join(outputPath, "icdMapping.pkl"), "wb") as f:
#     pickle.dump(helper.icdDict, f)

In [23]:
print(helper.icdDict)
print(helper.tok2id)
print(helper.max_length)

{'456': 2, '45': 0, '18': 1, '123': 5, '486': 4, '4240': 3}
{'cat': 2, 'horse': 3, 'CASE:aa': 5, 'welcometonightvale': 10, 'deadpool': 4, '</s>': 11, 'CASE:Aa': 8, 'dog': 1, 'CASE:AA': 6, 'UUUNKKK': 10, '<s>': 9, 'CASE:aA': 7}
3


In [24]:
train_raw

[(['dog', 'dog'], ['']),
 (['deadpool', 'welcomeToNightVale'], ['']),
 (['deadpool', 'deadpool'], ['']),
 (['cat', 'dog', 'horse'], ['']),
 (['horse'], ['']),
 (['dog', 'dog', 'dog', 'dog'], ['']),
 (['dog', 'dog', 'dog', 'cat'], [''])]

In [25]:
train

[([[1, 5], [1, 5]], array([ 0.,  0.,  0.,  0.,  0.,  0.])),
 ([[4, 5], [10, 7]], array([ 0.,  0.,  0.,  0.,  0.,  0.])),
 ([[4, 5], [4, 5]], array([ 0.,  0.,  0.,  0.,  0.,  0.])),
 ([[2, 5], [1, 5], [3, 5]], array([ 0.,  0.,  0.,  0.,  0.,  0.])),
 ([[3, 5]], array([ 0.,  0.,  0.,  0.,  0.,  0.])),
 ([[1, 5], [1, 5], [1, 5]], array([ 0.,  0.,  0.,  0.,  0.,  0.])),
 ([[1, 5], [1, 5], [1, 5]], array([ 0.,  0.,  0.,  0.,  0.,  0.]))]

In [26]:
print(embeddings[helper.tok2id['welcometonightvale'], 1:5])
print(embeddings[helper.tok2id['deadpool'], 1:5])
print(embeddings[helper.tok2id['UUUNKKK'], 1:5])

[-1.18571389  0.13177936 -0.98973244  0.38124523]
[ 0.004422  0.006439 -0.004394  0.009065]
[-1.18571389  0.13177936 -0.98973244  0.38124523]


In [101]:
train_raw

[(['the', 'dog', 'dog', 'horse'], ['4240', '486']),
 (['cat', 'cat', 'cat', 'horse'], ['4240', '486', '123', '45', '18']),
 (['deadpool', 'deadpool'], ['4240']),
 (['cat', 'dog', 'horse'], ['486']),
 (['horse'], ['123', '456']),
 (['dog', 'dog', 'dog', 'dog'], ['4240', '486']),
 (['dog', 'dog', 'dog', 'cat'], ['4240', '486'])]

In [129]:
helper.tok2id

{'</s>': 11,
 '<s>': 9,
 'CASE:AA': 6,
 'CASE:Aa': 8,
 'CASE:aA': 7,
 'CASE:aa': 5,
 'UUUNKKK': 10,
 'cat': 3,
 'deadpool': 2,
 'dog': 1,
 'horse': 4,
 'welcometonightvale': 5}

In [103]:
helper.tok2id

{'</s>': 11,
 '<s>': 9,
 'CASE:AA': 6,
 'CASE:Aa': 8,
 'CASE:aA': 7,
 'CASE:aa': 5,
 'UUUNKKK': 10,
 'cat': 2,
 'deadpool': 4,
 'dog': 1,
 'horse': 3,
 'the': 5}

In [111]:
helper.icdDict

{'123': 5, '18': 1, '4240': 3, '45': 0, '456': 2, '486': 4}

In [105]:
helper.icdDict

{'123': 5, '18': 1, '4240': 3, '45': 0, '456': 2, '486': 4}

In [14]:
import os
import pickle
import numpy as np
import tensorflow as tf
# from scipy import stats
import tensorflow as tf
# from sklearn.metrics import roc_auc_score
# import sklearn.metrics
from sklearn import metrics
import scipy
import matplotlib.pyplot as plt
import time
import logging
import sys

In [21]:
saver = tf.train.import_meta_graph('tempDelete/bestModel.meta')

# We can now access the default graph where all our metadata has been loaded
graph = tf.get_default_graph()

# Finally we can retrieve tensors, operations, collections, etc.
# returns tensors with these given namres
# global_step_tensor = graph.get_tensor_by_name('loss/global_step:0')
# train_op = graph.get_operation_by_name('loss/train_op')
# hyperparameters = tf.get_collection('hyperparameters')

In [25]:
graph

In [23]:
# global_step_tensor = graph.get_tensor_by_name('loss/global_step:0')
# train_op = graph.get_operation_by_name('loss_function')
# hyperparameters = tf.get_collection('hyperparameters')

KeyError: "The name 'loss_function' refers to an Operation not in the graph."

In [27]:
with tf.Session() as sess:
    # To initialize values with saved data
    saver.restore(sess, 'tempDelete/bestModel.data-00000-of-00001')
    print(sess.run(global_step_tensor)) # returns 1000

DataLossError: Unable to open table file tempDelete/bestModel.data-00000-of-00001: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[Node: save_3/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_3/Const_0, save_3/RestoreV2/tensor_names, save_3/RestoreV2/shape_and_slices)]]

Caused by op 'save_3/RestoreV2', defined at:
  File "/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/traitlets/config/appl

In [40]:
with tf.Session() as sess:
#     new_saver = tf.train.import_meta_graph('my-save-dir/my-model-10000.meta')
#     new_saver.restore(sess, 'my-save-dir/my-model-10000')
    new_saver = tf.train.import_meta_graph('tempDelete/bestModel-2.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('tempDelete/'))
    y_last = tf.get_collection('y_last')
    print(y_last)
#     for v in y_last:
#         print('hello')
#         v_ = sess.run(v)
#         print(v_)

[<tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'RNN_OutsideCell/Add:0' shap